# AVL Tree

## Implementing a binary search tree first

In [345]:
    
# 30,25,35,20,15,5,10,50,60,70,65
b = AVL_tree(30)
b.add(25)
b.add(35)
b.add(20)
b.add(15)
b.add(5)
b.add(10)
b.add(50)
b.add(60)
b.add(70)
b.add(65)

print(b)

LL condition
LL condition
LR condition
RR condition
RR condition
RL condition
        5(1)
    10(2)
        15(1)
20(4)
            25(1)
        30(2)
            35(1)
    50(3)
            60(1)
        65(2)
            70(1)



In [512]:
b = AVL_tree(10)

b.add(40)

b.add(50)
b.add(60)
b.add(70)
b.add(5)

b.add(45)
b.add(80)




print(b)
b.delete_value(80)
print(b)


RR condition
RR condition
        5(1)
    10(2)
40(4)
            45(1)
        50(2)
    60(3)
        70(2)
            80(1)

        5(1)
    10(2)
40(4)
            45(1)
        50(2)
    60(3)
        70(1)



In [447]:
not None

True

In [506]:
class Tree_Node():
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        
        # for AVL tree
        self.level = 1
        
class AVL_tree():
    def __init__(self, value):
        # 紀錄 AVL tree 的 root，方便使用
        self.root = Tree_Node(value)
    
    def get_balance(self, node):
        # for recurrsive，被查詢的node如果是None，則沒有 unbalance的問題，回傳 0
        if node is None:
            return 0
        # 回傳被查詢node 的 left/right level差異
        else:
            return self.get_level(node.left) - self.get_level(node.right)
        
    def get_level(self, node):
        # 如果傳入查詢的 node (可能是 left/right) 是 None，則回傳 0
        if node is None:
            return 0
        # 如果不是 none，則回傳該node的 level
        else:
            return node.level
        
    def delete_value(self, value, root_node=None):
        # 需要指定 root_node，因為方便 recurssive 迴圈使用，也可以用在指定要刪除的 node
        if root_node is None:
            # 從外部呼叫的一次迴圈的話，先測試此 value是否在 AVL tree 裡面
            # 第一次測試就好，避免每次 recursive 都測試一次
            if self.check_precence(value)==False:
                raise Exception('No precence of value')
            root_node = self.root
            
        # 通過上面的判斷式子到這裡的時候，代表 delete_value 一定在 AVL tree 裡面
        # 不斷的呼叫刪除node，賦值給上一個 parent
        if value<root_node.value:
            root_node.left = self.delete_value(value, root_node.left)
        elif value>root_node.value:
            root_node.right = self.delete_value(value, root_node.right)
        
        # 找到 value後，以下進行 刪除node
        # 並把被刪除的 node的children，依據不同情況回傳給上一階段的 parent
        else:
            # deleted node is leaf
            if root_node.right is None and root_node.left is None:
                # 其實如果這段不寫，leaf node兩側都是 None，會落到下面一個，也是會 return None
                return None
            # deleted node has one child
            elif root_node.left is None:
                temp_node = root_node.right
                root_node = None
                # 會把這個經過處理過後的 node，回傳給上面的賦值，給 parent_node 的 left or right
                return temp_node
            elif root_node.right is None:
                temp_node = root_node.left
                root_node = None
                return temp_node
            # deleted node has 2 children
            # 找到 deleted node.right 中最小的那個 node，命為 successor
            successor_node = self.get_min_value_node(root_node.right)
            # 把 successor node 的值賦予 root node
                # 這樣不用動到 root_node的連結，比較方便
                # 如果要動，要 root_node 的 parent以及child都需要變動，會是大工程
            assign_value = successor_node.value
            root_node.value = assign_value

            # 然後再呼叫一次本 method，從 root_node.right 開始尋找，刪掉 successor node
                # 把 root_node.right 想成一串獨立的 AVL tree
                # 將 root_node.right 賦值為刪除掉 successor value 後的 Tree
            # 這裡必須寫成能指定刪除的 node的 function
                # 如果寫成從 self.root開始找，刪除特定value，而不是node的話，在此AVL tree中，可能會有 value相同的而被刪錯
                # 因為需要有 root_node.right/left = delete_value(delete_value, root_node.right)
                # 這樣才能將 被刪除的上一個 node和被刪除的 node 做連結，利用 recursive 的特性
            root_node.right = self.delete_value(successor_node.value, root_node.right)
            
        # 上面的部分為刪除 node
        
        ##########################
        
        # 下面的部分是刪除掉node之後，判斷是否需要進行 rotation        
        # recusive 會走到這邊來的時候，都是走完其下的某個刪除步驟之後，才會到這邊
            # 因為 被刪除的 node，會在上面的部分 return
            # 經由 root_node.r/l = self.delete_value() 賦值，然後走到這一層
        # 這裡也利用 recrusive in stack 的特別之處，一步一步回去計算此時的 root node是否需要 rotation
        # 第一個跑到這邊的，是 被刪掉後的上一個node 
        # 如果需要 rotation，將會把 這個rotation後的這個分支，在最後 return 給上更一階段(grand_parent_node)
        
        # 重新沿路回去計算 level，看看是否 balance
        root_node.level = max(self.get_level(root_node.left), self.get_level(root_node.right)) +1
        balance_condition = self.get_balance(root_node)

        # LL condition
        # if root_node.left is None -> then get_balance is 0
        # 這裡的 get_balance的判斷式包含等於，這點很重要
            # 因為當root_node不平衡的時候，要往其下判斷4種condition
            # 有時候要被拿取的arm的兩邊，互為 balance，如果不包含等於，就不會繼續執行rotation了
            # 但被拿取的兩arm如果 balance 的話要拿哪邊，會依照code的先後順序 
        if balance_condition>1 and self.get_balance(root_node.left)>=0:
            new_root = self.right_rotation(root_node)
            # 如果是 root imbalance，要 reset self.root:
            if root_node == self.root:
                self.root = new_root
            # 傳回去給 更上一個node的分支 (grand_parent_node)
            return new_root
        # RR condition
        elif balance_condition<-1 and self.get_balance(root_node.right)<=0:
            new_root = self.left_rotation(root_node)
            if root_node==self.root:
                self.root =new_root
            return new_root
        # LR condition
        elif balance_condition>1 and self.get_balance(root_node.left)<=0:
            # 先將 root_node.left 做 left rotation，後將
            root_node.left = self.left_rotation(root_node.left)
            new_root = self.right_rotation(root_node)
            
            if root_node == self.root:
                self.root = new_root
            
            return new_root
        # RL condition
        elif balance_condition<-1 and self.get_balance(root_node.right)>=0:
            root_node.right = self.right_rotation(root_node.right)
            new_root = self.left_rotation(root_node)
            
            if root_node == self.root:
                self.root = new_root
            
            return new_root
        
        # 當tree 需要 rotation的時候，會在上面就return給其上一個node.right/left了
        ############
        # 當刪除node之後，往上歷遍所有的node都再不需要 rotaion，就在這裏回傳整串給上一個 node.right/left
        
        return root_node
                
                
    def check_precence(self, value):
        node = self.root
        
        while True:
            if value==node.value:
                return True
            elif value<node.value:
                node = node.left
            else:
                node = node.right
            if node is None:
                return False
                
    
    # get minimal value node, a helper method
    def get_min_value_node(self, node):
        if node is None or node.left is None:
            return node
        else:
            return self.get_min_value_node(node.left)

    

    def update_level(self, current_node=None):
        # 進行 post-order traversal，update 每一個 node 的 height/level
        if current_node is None:
            current_node = self.root

        # leaf node
        if current_node.left is None and current_node.right is None:
            current_node.level = 1
        # 只有 right/left 有node 的情況
        elif current_node.left is None:
            current_node.level = self.update_level(current_node.right)+1
        elif current_node.right is None:
            current_node.level = self.update_level(current_node.left) +1
        # 兩邊都有 node 的情況
        else:
            current_node.left.level = self.update_level(current_node.left)
            current_node.right.level = self.update_level(current_node.right)
            current_node.level = max(current_node.left.level,current_node.right.level) +1
        return current_node.level
        
    # call recursively 
    def add(self, value, root_node=None):
        if root_node is None:
            root_node = self.root
        if value<=root_node.value:
            # 如果左側沒有 node，則加入
            if root_node.left is None:
                root_node.left = Tree_Node(value)
            # 如果左側已經有 node，則在左側以下加入node
            else:
                # 則是 AVL tree 的一個重要的地方。
                # 一般 binary tree 只要 self.add(value, root_node.left) 即可增加連結，
                # 但是 AVL tree rotation 的時候，會改變 先前 node對下一個node(new_root)的連結
                # 如果沒有 root_node.left 賦值，則無法獲得rotation後的新的連結
                # 因此， self.add()必須要 return new_root
                root_node.left = self.add(value, root_node.left)
        else:
            if root_node.right is None:
                root_node.right = Tree_Node(value)
            else:
                root_node.right = self.add(value, root_node.right)
                
        # 每被call一次，中間經過的node都要加一個level
        # 而且是 stack加回來，所以最靠近 leaf node的那個會先加
        # 然後就依序往root 方向 update level 數值，一路加回來
        # 所以需要一個 get_level function，方便處理如果 child is None 的狀況
        root_node.level = max(self.get_level(root_node.left), self.get_level(root_node.right)) +1
        
        # 加完 node level 之後，查詢是否 disbalance
        # >1 -> left>right -> left condition
        # -- root.left balance >0 / value<root.left.value -> left-left
        # -- root.left balance <0 -> left-right
        # 0,1,-1 -> balance
        # <-1 -> right condition
        # -- root.right balance >0 -> right-left
        # -- root.right balance <0 -> right-right
        balance_condition = self.get_balance(root_node)
        
        # Left-left condition
        if balance_condition>1 and value<root_node.left.value:
            print('LL condition')
            new_root = self.right_rotation(root_node)
            # 上面有解釋為什麼要 return new_noot
            return new_root
        # Left-right condition
        elif balance_condition>1 and value>root_node.left.value:
            print('LR condition')
            # 針對 root.left 行left rotation，回傳該rotation後的rotated_root
            rotated_root = self.left_rotation(root_node.left) 
            # 將root.left rotation後的 rotated_root，接上
            root_node.left = rotated_root
            # 再針對new_root行 right rotation
            new_root = self.right_rotation(root_node)
            return new_root
        # Right-right condition
        elif balance_condition<-1 and value>root_node.right.value:
            print('RR condition')
            new_root = self.left_rotation(root_node)
            return new_root
        # Right-left condition
        elif balance_condition<-1 and value<root_node.right.value:
            print('RL condition')
            # 針對 root.right 行 right rotation
            rotated_root = self.right_rotation(root_node.right)
            root_node.right = rotated_root
            new_root = self.left_rotation(root_node)
            return new_root
        
        # 如果都沒有 rotation，return root_node
        # 這裡的 return node 非常重要，因為要讓上一個 node 知道要連接到這個 node
        
        return root_node


    def right_rotation(self, disbalance_node):
        new_root = disbalance_node.left  # 將原root.left設為新root
        disbalance_node.left = new_root.right # 將新 root.right搬到原root.left的空缺
        new_root.right = disbalance_node # 新root.right 設定為原root
        
        disbalance_node.level = max(self.get_level(disbalance_node.left), self.get_level(disbalance_node.right))+1
        new_root.level = max(self.get_level(new_root.left), self.get_level(new_root.right))+1
        
        # 如果 self.root 是 disbalance 的node，則要 update self.root為new_root
        if disbalance_node==self.root:
            self.root = new_root
        return new_root
    
    def left_rotation(self, disbalance_node):
        new_root = disbalance_node.right # 將原root.right設為新root
        disbalance_node.right = new_root.left
        new_root.left = disbalance_node
        
        disbalance_node.level = max(self.get_level(disbalance_node.left), self.get_level(disbalance_node.right))+1
        new_root.level = max(self.get_level(new_root.left), self.get_level(new_root.right))+1
        
        if disbalance_node==self.root:
            self.root = new_root
        return new_root
    
        
    def __str__(self, level=0, node=None):
        if node is None:
            node = self.root
        if node is None:
            return ''
        return_value = '    '*level + f"{str(node.value)}({node.level})" + '\n'
        left_value = '' if node.left is None else self.__str__(level+1, node.left)
        right_value = '' if node.right is None else self.__str__(level+1, node.right)
        return_value = left_value + return_value + right_value
        return return_value
    
    
    # traversal by using queue
    def levelOrder(self, node=None):
        if node is None:
            node = self.root
        queue = Q()
        queue.enQ(node)
        return_list = []
        
        while queue.is_empty() is False:
            # 拿出一個 node
            current_node = queue.deQ()
            return_list.append(str(f"{current_node.value}({current_node.level})"))
            # print(f"Node:{current_node.value}, Level:{self.count_level(current_node)}")
            
            # 把兩邊children 加上去
            if current_node.left is not None:
                queue.enQ(current_node.left)
            if current_node.right is not None:
                queue.enQ(current_node.right)
            
        return ' '.join(return_list)


    

    


In [ ]:
class Q():
    def __init__(self):
        self.head = None
        self.tail = None
        
    def enQ(self, value):
        new_node = N(value)
        
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node
            
    def __iter__(self):
        current_node = self.head
        while current_node:
            yield current_node
            current_node = current_node.next
            
    def __str__(self):
        ret_str = [str(n.value) for n in self if n.value is not None]
        return ' '.join(ret_str)
            
    def is_empty(self):
        if self.head is None:
            return True
        else:
            return False
    
    def deQ(self):
        if self.is_empty():
            return None
        else:
            ret_value = self.head.value
            self.head =self.head.next
            return ret_value
        
class N():
    def __init__(self, value):
        self.value = value
        self.next = None
        



## Other

In [ ]:
    # my solution
    def count_level(self, current_node=None):
        # initial call, set as self
        if current_node is None:
            current_node = self
            
        # right arm, left arm, 取高的, 加上1
        # 從最末端往回推算。leaf node -> return 0 -> 依序往上加 
        right = 0 if current_node.right is None else self.count_level(current_node.right)
        left = 0 if current_node.left is None else self.count_level(current_node.left)
        
        return max(left,right)+1
    
        

    def count_level(self, current_node=None):
        # 進行 post-order traversal，update 每一個 node 的 height/level
        if current_node is None:
            current_node = self

        # leaf node
        if current_node.left is None and current_node.right is None:
            current_node.level = 1
        # 只有 right/left 有node 的情況
        elif current_node.left is None:
            current_node.level = current_node.right.count_level()+1
        elif current_node.right is None:
            current_node.level = current_node.left.count_level() +1
        # 兩邊都有 node 的情況
        else:
            current_node.left.level = current_node.left.count_level()
            current_node.right.level = current_node.right.count_level()
            current_node.level = max(current_node.left.level,current_node.right.level) +1
        return current_node.level

## Not working, delete node


In [ ]:
    def delete_value(self, search_value):
        # 先確認 value 有在裡面，且找到該 node的 parent
        parent_node = None
        current_node = self.root  # start from root
        
        while True:
            # if the tree is empty or has not found value
            if current_node is None:
                raise Exception('The value is not found')
                return None
            elif search_value==current_node.value:
                break
            elif search_value<current_node.value:
                parent_node = current_node
                current_node = current_node.left
            else:
                parent_node = current_node
                current_node = current_node.right
        # found parent_node and current_node

        # to delete
        # current_node is a leaf
        if current_node.left is None and current_node.right is None:
            # 刪掉唯一一個 node:
            if current_node==self.root:
                self.root.value=None
                self.root = None
            else:
                if current_node.value<parent_node.value:
                    parent_node.left = None
                else:
                    parent_node.right = None
        # current_node 有一邊是空的
        elif current_node.left is None:
            # 如果要刪掉 root_node
            if current_node==self.root:
                self.root = current_node.right
                current_node.right = None
            # 這裡 elif 很重要，不是 if，因為 parent_node 可能為 None(if current is root)
            elif current_node.value<parent_node.value:
                parent_node.left = current_node.right
            else:
                parent_node.right = current_node.right
        elif current_node.right is None:
            # 如果要刪掉 root_node
            if current_node==self.root:
                self.root = current_node.left
                current_node.left = None
            elif current_node.value<parent_node.value:
                parent_node.left = current_node.left
            else:
                parent_node.right = current_node.left
        # current_node 兩邊都有 node，皆不為 None
        else:
            # 從 current_node.right 中選出最小的那個來替代
            # pick candidate
            successor_node = current_node.right
            while True:
                if successor_node.left is None:
                    break
                successor_node = successor_node.left
                
            # succssor_node 最多只會有一個 child
            # 如果 successor_node 沒有 child
            
            # 用 successor node 的value取代掉原 node 的 value就好，不用搬動
            # 如果刪掉 root node -> 沒差
                # 因為不會“刪除” current_node，只是改變 value
                # 所以不需要改變 parent node的指標
                # 也不需改變 root_node 的內容
                
            assign_value = successor_node.value
            # 直接呼叫刪掉value值為 successor node的那個node, 最多只會跑到單邊有value那邊
            # 因為真正被刪掉的是該 node，所以會發生不平衡的地方，也是在 successor node 的 parent
            self.delete_value(successor_node.value)
            current_node.value = assign_value
            
        # update level
        self.update_level()
        
        # see if parent_node is unbalanced
        # 注意，如果原 parent node is None (change root)者不需要處理
        # 因為 2 children root 真正被刪掉的不是 root
        # 1 child root 直接接直接的 child，無須考慮 disbalance
            # 在add to disbalance之前就會先被處理掉了
        # 0 child root 直接被刪掉
        if parent_node is None:
            return True
        
        
        
        # search for grand_parent_node
        queue = Q()
        queue.enQ(self.root)
        return_list = []
        
        while queue.is_empty() is False:
            # 拿出一個 grand_parent_node
            grand_parent_node = queue.deQ()
            
            # for detect if unbalanced node is self.root
            balance_of_gp_node = self.get_balance(grand_parent_node)
            if balance_of_gp_node>1 or balance_of_gp_node<-1:
                root_node = self.root
                print('root')
                break
            
            right_balance_value = self.get_balance(grand_parent_node.right)
            left_balance_value = self.get_balance(grand_parent_node.left)
            
            if right_balance_value>1 or right_balance_value<-1:
                root_node = grand_parent_node.right
                status = 'right'
                break
            elif left_balance_value>1 or left_balance_value<-1:
                root_node = grand_parent_node.left
                status = 'left'
                break
            # node 加上去
            if grand_parent_node.left is not None:
                queue.enQ(grand_parent_node.left)
            if grand_parent_node.right is not None:
                queue.enQ(grand_parent_node.right)
        else:
            # no disbalance occurs
            return True
    
        balance_condition = self.get_balance(root_node)
        
        print('balance condition=', balance_condition )
        
        # Left-left condition
        # parent node left condition, and child node left level is greater
        if balance_condition>1 and self.get_balance(root_node.left)>0:
            print('LL condition')
            new_root = self.right_rotation(root_node)
            # 上面有解釋為什麼要 return new_noot            
        # Left-right condition
        elif balance_condition>1 and self.get_balance(root_node.left)<0:
            print('LR condition')
            # 針對 root.left 行left rotation，回傳該rotation後的rotated_root
            rotated_root = self.left_rotation(root_node.left) 
            # 將root.left rotation後的 rotated_root，接上
            root_node.left = rotated_root
            # 再針對new_root行 right rotation
            new_root = self.right_rotation(root_node)
        # Right-right condition
        elif balance_condition<-1 and self.get_balance(root_node.right)<0:
            print('RR condition')
            new_root = self.left_rotation(root_node)
        # Right-left condition
        elif balance_condition<-1 and self.get_balance(root_node.right)>0:
            print('RL condition')
            # 針對 root.right 行 right rotation
            rotated_root = self.right_rotation(root_node.right)
            root_node.right = rotated_root
            new_root = self.left_rotation(root_node)
            
        # 如果 被替換的是root node，那沒有 grand_parent 可言
        # 需要替換掉 self.root
        if root_node==self.root:
            self.root = new_root
        else:
            # 如果被替換的不是 root node
            # assign new root to grand_parent_noode
            if status=='right':
                grand_parent_node.right = new_root
            else:
                grand_parent_node.left = new_root

        # update level again
        self.update_level()
            
        return True